In [2]:
pip install httpx


  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Note: you may need to restart the kernel to use updated packages.


In [56]:
import httpx
import json
from typing import Dict
from urllib.parse import quote


In [58]:
def scrape_post(ig_doc_id ,url_or_shortcode: str) -> Dict:
    """Scrape single Instagram post data"""
    print(f"Scraping: {url_or_shortcode}, id: {ig_doc_id}")
    
    if "http" in url_or_shortcode:
        shortcode = url_or_shortcode.split("/p/")[-1].split("/")[0]
    else:
        shortcode = url_or_shortcode

    variables = quote(json.dumps({
        'shortcode':shortcode,'fetch_tagged_user_count':None,
        'hoisted_comment_id':None,'hoisted_reply_id':None
    }, separators=(',', ':')))
    body = f"variables={variables}&doc_id={ig_doc_id}"
    url = "https://www.instagram.com/graphql/query"

    result = httpx.post(
        url=url,
        headers={"content-type": "application/x-www-form-urlencoded"},
        data=body
    )
    data = json.loads(result.content)
    
    print(data)
    
    # return data["data"]["xdt_shortcode_media"]
    return data

In [ ]:
# document id for content query:
#more post content: 
# IG_DOC_ID = "9195084877213487"
#more content
# IG_DOC_ID = "9507655305965624"



#relationships/related : 9480525715305126
# IG_DOC_ID = "9480525715305126"
#acc metadata: 9109150515847101
# IG_DOC_ID = "9109150515847101"
#reels: 9318096484942317
# IG_DOC_ID = "9318096484942317"

In [6]:
 ! pip install pytest-playwright
 ! playwright install

  Using cached pyee-12.1.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 7.5 MB/s eta 0:00:00a 0:00:01m
Using cached pyee-12.1.1-py3-none-any.whl (15 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Removing unused browser at /Users/chaewonbang/Library/Caches/ms-playwright/chromium_headless_shell-1155
124.1 MiB [                    ] 0% 0.0s124.1 MiB [                    ] 0% 13.8s124.1 MiB [                    ] 0% 11.4s124.1 MiB [                    ] 0% 9.3s124.1 MiB [                    ] 0% 14.3s124.1 MiB [                    ] 0% 14.5s124.1 MiB [                    ] 0% 12.2s124.1 MiB [                    ] 1% 12.6s124.1 MiB [                    ] 1% 12.8s124.1 MiB [                    ] 1% 14.5s124.1 MiB [                    ] 1% 12.7s124.1 MiB 

In [59]:
import asyncio
from playwright.async_api import async_playwright
import re
import json

In [60]:
#This is a dynamic environment that stays open after execution

# Global references for easy access across notebook cells
playwright = await async_playwright().start()
device = playwright.devices["Pixel 5"]
browser = await playwright.chromium.launch(headless=False)  # Browser will stay open
context = await browser.new_context(**device)
page = await context.new_page()
# You now have persistent global references:
# playwright, browser, context, page

In [9]:
! pip install python-dotenv

In [61]:
from dotenv import load_dotenv
#load the .env file
# get USERNAME and PASSWORD
load_dotenv(override=True)
import os
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")

In [62]:
username

'databot363792934'

In [63]:
#signon doesn't work rn

import asyncio
from playwright.async_api import TimeoutError

async def signon(page, username, password):
    try:
        await page.goto("https://www.instagram.com/accounts/login/", wait_until="networkidle")

        
        await page.wait_for_selector('input[name="username"]', timeout=10000)
        await asyncio.sleep(0.5)
        await page.fill('input[name="username"]', username)
        await asyncio.sleep(0.5)
        await page.fill('input[name="password"]', password)
        await asyncio.sleep(0.5)

        await page.click('button[type="submit"]')
        await asyncio.sleep(1)

        # Wait for navigation or check for login errors
        await page.wait_for_selector('nav', timeout=10000)  # Wait for main navigation bar as successful login indicator

        # Verify successful login by checking URL or user-specific elements
        if "login" in page.url:
            raise ValueError("Login failed: Incorrect username or password.")

        print("✅ Successfully logged in")
        
        return

    except TimeoutError:
        raise TimeoutError("Login timed out. Check credentials or network connection.")
    
    

In [64]:
#it says it fails but it works
await signon(page, username, password)

ValueError: Login failed: Incorrect username or password.

In [65]:
from random import randint

In [66]:
import asyncio
import re
import json
from random import randint
from datetime import datetime

async def get_total_posts(page):
    """Extracts the total number of posts from the user's profile."""
    total_posts = await page.evaluate("""
        () => {
            let element = document.querySelector('header section ul li span');
            return element ? parseInt(element.innerText.replace(',', '')) : null;
        }
    """)
    return total_posts if total_posts else float('inf')

async def get_post_date(post_element):
    """Extracts the post date from a post element."""
    timestamp = await post_element.evaluate("""
        node => {
            let timeElem = node.querySelector('time');
            return timeElem ? timeElem.getAttribute('datetime') : null;
        }
    """)
    return datetime.fromisoformat(timestamp) if timestamp else None

MAX_SCROLLS = 10

async def scroll_to_load_all_posts(page, max_scrolls=MAX_SCROLLS):
    """Scrolls until a post from before January 2023 is found or max scroll limit is reached."""
    all_posts = set()
    previous_count = 0
    scroll_attempts = 0

    print(f"🔄 Starting scrolling process... (Max {max_scrolls} attempts)")
    
    while scroll_attempts < max_scrolls:
        posts = await page.query_selector_all("div._aagu")
        if not posts:
            print("⚠️ No posts found in current DOM snapshot.")
            break

        # Check last post's date to determine if we should stop scrolling
        last_post = posts[-1]
        last_post_date = await get_post_date(last_post)

        if last_post_date:
            print(f"🕒 Last post date: {last_post_date}")
            if last_post_date < datetime(2023, 1, 1):
                print(f"🛑 Found post from {last_post_date}, stopping scrolling.")
                break  # Stop scrolling if an old post is found

        for post in posts:
            parent = await post.evaluate_handle("node => node.closest('a')")
            if parent:
                href = await parent.get_attribute('href')
                if href and '/p/' in href:
                    if href not in all_posts:  # Only process new posts
                        post_date = await get_post_date(post)
                        if post_date:
                            print(f"📅 Found new post: {href} | Date: {post_date}")
                        all_posts.add(href)

        if len(all_posts) == previous_count:
            print(f"✅ Scrolling stopped after {scroll_attempts} attempts. Total unique posts found: {len(all_posts)}")
            break  # Stop if no new posts are loaded

        previous_count = len(all_posts)
        scroll_attempts += 1

        # Scroll and wait
        print(f"📜 Scrolling attempt {scroll_attempts}... (Loaded posts: {len(all_posts)})")
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight);")
        await asyncio.sleep(3 + 0.1 * randint(0, 10))

    print(f"✅ Finished scrolling. Total posts loaded: {len(all_posts)}")
    return all_posts


In [73]:
import asyncio
import re
import json
from datetime import datetime

MAX_SCROLLS = 10
SCROLL_DELAY = 2  # Time between scrolls in seconds
testname = "daisychoiii"
profile_url = f"https://www.instagram.com/{testname}/"

### ========================= UTILITY FUNCTIONS ========================= ###
async def get_total_posts(page):
    """Extracts the total number of posts from the user's profile."""
    total_posts = await page.evaluate("""
        () => {
            let element = document.querySelector('header section ul li span');
            return element ? parseInt(element.innerText.replace(',', '')) : null;
        }
    """)
    return total_posts if total_posts else float('inf')

async def get_post_date(post_element):
    """Extracts the post date from a post element."""
    timestamp = await post_element.evaluate("""
        node => {
            let timeElem = node.querySelector('time');
            return timeElem ? timeElem.getAttribute('datetime') : null;
        }
    """)
    try:
        return datetime.fromisoformat(timestamp) if timestamp else None
    except Exception:
        return None

async def intercept_response(response, intercepted_posts, response_received_flag):
    """Intercepts post data from Instagram API responses."""
    endpoint_pattern = re.compile(r"/api/v1/media/\d+/info/")
    if endpoint_pattern.search(response.url) and response.status == 200:
        try:
            data = await response.json()
            media_id = data.get('items', [{}])[0].get('id', 'unknown')
            intercepted_posts[media_id] = data
            response_received_flag[0] = True
            print(f"✅ Intercepted post {media_id} ({len(intercepted_posts)} collected)")
        except Exception as e:
            print(f"❌ Failed to parse intercepted data: {e}")

### ========================= SCROLLING FUNCTION ========================= ###
async def scroll_to_load_all_posts(page, max_scrolls=MAX_SCROLLS):
    """Scrolls dynamically until the *last* visible post is from before January 2023 or max_scrolls is reached."""
    all_posts = set()
    previous_count = 0
    scroll_attempts = 0

    print(f"🔄 Starting scrolling process... (Max {max_scrolls} attempts)")

    while scroll_attempts < max_scrolls:
        posts = await page.query_selector_all("div._aagu")

        if not posts:
            print("⚠️ No posts found.")
            break

        # # Check the date of the last visible post only
        # last_post = posts[-1]
        # last_post_date = await get_post_date(last_post)

        # if last_post_date:
        #     print(f"🕒 Last post on page: {last_post_date}")
        #     if last_post_date < datetime(2023, 1, 1):
        #         print(f"🛑 Last post is before 2023. Stopping scroll.")
        #         break

        # Collect post links
        for post in posts:
            parent = await post.evaluate_handle("node => node.closest('a')")
            if parent:
                href = await parent.get_attribute('href')
                if href and '/p/' in href and href not in all_posts:
                    all_posts.add(href)

        if len(all_posts) == previous_count:
            print(f"✅ No new posts loaded after scroll {scroll_attempts}. Total: {len(all_posts)}")
            break

        previous_count = len(all_posts)
        scroll_attempts += 1

        print(f"📜 Scrolling attempt {scroll_attempts}... (Collected posts: {len(all_posts)})")
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight);")
        await asyncio.sleep(SCROLL_DELAY)

    print(f"✅ Finished scrolling. Total posts collected: {len(all_posts)}")
    return all_posts

### ========================= NAVIGATION & DATA COLLECTION ========================= ###
async def navigate_and_collect(page, all_post_links):
    """Visits each post URL and collects data via API interception."""
    intercepted_posts = {}
    response_received_flag = [False]

    print(f"🟢 Total unique posts queued for processing: {len(all_post_links)}")

    # Register a single persistent response handler
    page.on("response", lambda response: asyncio.create_task(intercept_response(response, intercepted_posts, response_received_flag)))

    for i, href in enumerate(all_post_links, start=1):
        post_url = f"https://www.instagram.com{href}"
        print(f"\n({i}/{len(all_post_links)}) 📌 Navigating to: {post_url}")

        response_received_flag[0] = False  # Reset before each visit

        await page.goto(post_url, wait_until="networkidle")
        await asyncio.sleep(1)  # Allow responses to load

        # Wait dynamically for response interception (max 5 seconds)
        for _ in range(50):
            if response_received_flag[0]:
                break
            await asyncio.sleep(0.1)

        print("🔄 Returning to profile page...\n")
        await page.goto("about:blank")
        await page.goto(profile_url, wait_until="networkidle")
        await asyncio.sleep(1)

    print(f"\n🎉 **Scraping complete! Collected {len(intercepted_posts)} posts.**")
    return intercepted_posts

### ========================= MAIN SCRAPING FUNCTION ========================= ###
async def scrape_profile_posts(profile_url: str, page):
    """Orchestrates the entire scraping process."""
    print(f"🚀 Navigating to profile: {profile_url}")
    await page.goto(profile_url, wait_until="networkidle")
    await asyncio.sleep(1)

    total_posts = await get_total_posts(page)
    print(f"🔢 Total posts found on profile: {total_posts}")

    all_post_links = await scroll_to_load_all_posts(page)
    print(f"📸 Found {len(all_post_links)} unique posts after scrolling.")

    intercepted_posts = await navigate_and_collect(page, all_post_links)

    return intercepted_posts

### ========================= RUN SCRAPER ========================= ###
# Example usage:
data = await scrape_profile_posts(profile_url, page)
print(json.dumps(data, indent=2, default=str))


🚀 Navigating to profile: https://www.instagram.com/daisychoiii/
🔢 Total posts found on profile: 114
🔄 Starting scrolling process... (Max 10 attempts)
📜 Scrolling attempt 1... (Collected posts: 19)
📜 Scrolling attempt 2... (Collected posts: 28)
📜 Scrolling attempt 3... (Collected posts: 40)
📜 Scrolling attempt 4... (Collected posts: 51)
📜 Scrolling attempt 5... (Collected posts: 62)
✅ No new posts loaded after scroll 5. Total: 62
✅ Finished scrolling. Total posts collected: 62
📸 Found 62 unique posts after scrolling.
🟢 Total unique posts queued for processing: 62

(1/62) 📌 Navigating to: https://www.instagram.com/daisychoiii/p/DCn40cwRXlk/
🔄 Returning to profile page...


(2/62) 📌 Navigating to: https://www.instagram.com/daisychoiii/p/CmDUTGSLQ6L/
🔄 Returning to profile page...


(3/62) 📌 Navigating to: https://www.instagram.com/daisychoiii/p/Cl2_DWGurV8/
🔄 Returning to profile page...


(4/62) 📌 Navigating to: https://www.instagram.com/daisychoiii/p/CqUY-49Lgf4/
🔄 Returning to profile 

TimeoutError: Page.goto: Timeout 30000ms exceeded.
Call log:
  - navigating to "https://www.instagram.com/daisychoiii/p/CrEJJwrLto2/", waiting until "networkidle"


In [78]:
len(data)
# data÷

62

In [77]:
# save the data to a json file
with open('daisychoii_posts.json', 'w') as f:
    json.dump(data, f)